# read polymer data

In [1]:
# the table for trial analysis in xlsx is converted to csv for easier parsing

import csv
with open('IO.csv', 'r') as file:
    contents = csv.reader(file)
    
    data = []
    for row in contents:
        if row[0] != '' and row[0] != 'Matrix':
            data.append(row)

# get input (I) and output (O)

In [2]:
# input consists of processing temperature, screw, and, residence time (3-d vector)
# output consists of tensile strength (a scalar) 

I, O = [], []
for d in data:
    temperature = float(d[1][:-3])  # to remove OC
    if d[2] == '':
        screw = 0
    else:
        screw = float(d[2])
    if d[3] == '':
        residence = 0
    else:   
        residence = float(d[3])
    I.append([temperature, screw, residence])
    
    strength = float(d[4])
    O.append(strength)

# normalise output

In [3]:
maximum = max(O)
O = [o/maximum for o in O]

# [feedforward neural network](https://en.wikipedia.org/wiki/Feedforward_neural_network)

In [4]:
import torch, torch.nn as nn, torch.nn.functional as F

class FeedNet(nn.Module):
    
    def __init__(self):
        super(FeedNet, self).__init__()
        self.layer1 = nn.Linear(3, 16)
        self.layer2 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        out = F.relu(self.layer1(x))
        out = self.layer2(out)
        out = self.sigmoid(out)
        return out

# 80% train and 20% test

In [5]:
import random
full = range(len(I))
train = random.sample(full, int(0.8*len(I)))
test = list(set(full) - set(train))

# train the model

In [6]:
from tqdm import tqdm
criterion = nn.MSELoss()
I, O = torch.FloatTensor(I), torch.FloatTensor(O).unsqueeze(1)
FFNN = FeedNet()

optimiser = torch.optim.Adam(FFNN.parameters(), lr=0.001)
FFNN.train()
for _ in tqdm(range(100)):
    for i in train:
        optimiser.zero_grad()
        P = FFNN(I[i])
        loss = criterion(P, O[i])
        loss.backward()
        optimiser.step()

100%|██████████| 100/100 [00:03<00:00, 28.07it/s]


# test the model

In [14]:
FFNN.eval()
for i in test:
    P = FFNN(I[i])
    print("datapoint", i, ", actual:", round(float(O[i]*maximum), 3), ", and predicted:", round(float(P*maximum), 3))

datapoint 4 , actual: 1.87 , and predicted: 0.663
datapoint 37 , actual: 1.1 , and predicted: 0.275
datapoint 7 , actual: 2.5 , and predicted: 0.663
datapoint 8 , actual: 2.01 , and predicted: 0.663
datapoint 39 , actual: 10.7 , and predicted: 12.575
datapoint 43 , actual: 15.6 , and predicted: 12.575
datapoint 12 , actual: 1.06 , and predicted: 0.478
datapoint 15 , actual: 1.04 , and predicted: 0.478
datapoint 16 , actual: 1.14 , and predicted: 0.478


In [8]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run all cells',
    help_index : 'zz',
    handler : function (event) {
        IPython.notebook.execute_all_cells();
        return false;
    }}
);

<IPython.core.display.Javascript object>